In [2]:
import pandas  as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import selenium
import feedparser
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import requests as req
from bs4 import BeautifulSoup as bs    # ambos alias son cosa mia
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestRegressor

In [27]:
def machinelearning(feature):
    global column
    global predictdataraw
    global predictdata
    column = pd.read_csv(f'{feature}_AI_onlyv.csv').drop(['Unnamed: 0'], axis=1)
    predictdataraw = pd.read_csv(f'{feature}_AI_prediction.csv')
    predictdata = pd.read_csv(f'{feature}_AI_prediction.csv').drop(['Unnamed: 0'], axis=1)

In [28]:
machinelearning('equilibrium_temperature_[k]')

In [29]:
predictdata.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1383 entries, 0 to 1382
Data columns (total 5 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   equilibrium_temperature_[k]    0 non-null      float64
 1   orbital_period_[days]          1197 non-null   float64
 2   orbit_semi-major_axis_[au])    1194 non-null   float64
 3   stellar_radius_[solar_radius]  1188 non-null   float64
 4   insolation_flux_earth          90 non-null     float64
dtypes: float64(5)
memory usage: 54.1 KB


In [30]:
#pon aqui 
predictdata.drop(['equilibrium_temperature_[k]', 'insolation_flux_earth'] ,axis=1, inplace=True)
predictdata.dropna(inplace=True)

In [31]:
predictdata.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 985 entries, 0 to 1380
Data columns (total 3 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   orbital_period_[days]          985 non-null    float64
 1   orbit_semi-major_axis_[au])    985 non-null    float64
 2   stellar_radius_[solar_radius]  985 non-null    float64
dtypes: float64(3)
memory usage: 30.8 KB


In [32]:
column.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3498 entries, 0 to 3497
Data columns (total 5 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   equilibrium_temperature_[k]    3498 non-null   float64
 1   orbital_period_[days]          3498 non-null   float64
 2   orbit_semi-major_axis_[au])    3498 non-null   float64
 3   stellar_radius_[solar_radius]  3498 non-null   float64
 4   insolation_flux_earth          3498 non-null   float64
dtypes: float64(5)
memory usage: 136.8 KB


In [33]:
#pon aqui 
X = column.drop(['equilibrium_temperature_[k]', 'insolation_flux_earth'],axis=1)
y = column['equilibrium_temperature_[k]']
X

,orbital_period_[days],orbit_semi-major_axis_[au]),stellar_radius_[solar_radius]
0,0.736547,0.01544,0.94
1,8.462999,0.06450,0.70
2,2.290700,0.01850,0.17
3,1.508956,0.02752,1.11
4,2.828042,0.04016,1.12
...,...,...,...
3493,11.094286,0.10100,1.01
3494,6.252972,0.06200,0.73
3495,16.068638,0.11700,0.73
3496,41.194912,0.22000,0.73


In [34]:
y

0       1958.0
1        593.0
2        464.0
3       1898.0
4       1442.0
         ...  
3493     835.0
3494     823.0
3495     600.0
3496     439.0
3497    1157.0
Name: equilibrium_temperature_[k], Length: 3498, dtype: float64

In [35]:
def modelexecution(X, y, depth):
    X = StandardScaler().fit_transform(X)
    global y_test
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    global model1
    model1 = RandomForestRegressor(n_estimators=200, max_depth=depth, random_state=42)
    model1.fit(X_train, y_train)
    global y_pred_rf
    y_pred_rf = model1.predict(X_test)
    global MSE_RF
    MSE_RF = mean_squared_error(y_test, y_pred_rf, squared=False)
    global model2
    model2 = LinearRegression()
    model2.fit(X_train, y_train)
    global y_pred_lr
    y_pred_lr = model2.predict(X_test)
    global MSE_LR
    MSE_LR = mean_squared_error(y_test, y_pred_lr, squared=False)
    return MSE_LR, MSE_RF, y_test

In [36]:
modelexecution(X, y, 6)

(296.90268463233315,
 114.06500505973592,
 1650    1000.0
 2456    1043.0
 2232     687.0
 1945    1625.0
 309     1184.0
          ...  
 2670     428.0
 1253     376.0
 801      852.0
 2112    1104.0
 1374     727.0
 Name: equilibrium_temperature_[k], Length: 1050, dtype: float64)

In [37]:
y_test[:10]

1650    1000.0
2456    1043.0
2232     687.0
1945    1625.0
309     1184.0
2341    1117.0
1666     766.0
1681     876.0
1187    2550.0
325     1427.0
Name: equilibrium_temperature_[k], dtype: float64

In [38]:
#pon aqui 
mean = column['equilibrium_temperature_[k]'].mean()
mean

908.8644939965694

In [39]:
for i in y_pred_rf[:10]:
    print('{:.2f}'.format(i))

1049.31
994.19
722.80
1730.97
979.12
1118.70
802.95
948.49
2469.91
1103.06


In [40]:
final_pred = model1.predict(predictdata)

In [41]:
for i in final_pred[:]:
    print('{:.2f}'.format(i))

313.32
313.32
313.32
308.48
311.60
313.32
309.29
313.32
313.32
313.32
298.28
312.41
313.32
313.32
311.60
287.41
230.96
329.34
311.60
254.34
311.60
313.32
329.34
311.60
311.60
313.32
313.32
312.41
313.32
313.32
309.29
313.32
313.32
311.60
313.32
313.32
313.32
313.32
313.32
311.60
313.32
313.32
271.00
311.60
311.60
311.60
328.59
309.14
326.00
309.71
331.05
305.24
307.08
313.32
309.14
327.89
305.88
331.86
328.70
331.05
312.94
328.70
331.05
253.93
225.19
315.17
339.61
325.35
316.61
310.97
296.46
301.76
301.76
296.46
316.61
252.54
324.26
306.36
304.50
304.50
301.76
304.50
304.50
304.50
341.87
230.69
309.71
305.27
305.27
306.19
301.76
319.67
306.01
350.85
322.06
305.27
304.92
255.11
312.11
267.61
301.76
301.76
301.76
302.68
314.38
295.95
305.84
296.46
296.46
327.04
320.80
327.39
306.19
301.76
305.42
309.38
266.99
305.27
305.78
307.13
319.67
302.68
322.17
302.68
305.27
319.67
309.14
228.41
327.42
307.44
228.03
301.76
301.76
305.27
304.50
304.50
288.67
305.27
256.22
311.55
301.76
301.76
337.64

In [47]:
len(final_pred)

985

In [42]:
predictdataraw

,Unnamed: 0,equilibrium_temperature_[k],orbital_period_[days],orbit_semi-major_axis_[au]),stellar_radius_[solar_radius],insolation_flux_earth
0,0,NaN,326.030000,1.290000,19.00,NaN
1,1,NaN,516.219970,1.530000,29.79,NaN
2,2,NaN,185.840000,0.830000,11.00,NaN
3,3,NaN,1765.038901,2.773069,0.93,NaN
4,4,NaN,798.500000,1.660000,1.13,NaN
...,...,...,...,...,...,...
1378,4762,NaN,1919.000000,3.053000,1.10,NaN
1379,4827,NaN,11.280000,NaN,1.26,NaN
1380,4828,NaN,737.113100,1.640000,1.12,0.56
1381,5073,NaN,28.731000,NaN,0.86,NaN


In [43]:
predictdata.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 985 entries, 0 to 1380
Data columns (total 3 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   orbital_period_[days]          985 non-null    float64
 1   orbit_semi-major_axis_[au])    985 non-null    float64
 2   stellar_radius_[solar_radius]  985 non-null    float64
dtypes: float64(3)
memory usage: 30.8 KB


In [51]:
#pon aqui la mas corta feature
algo = predictdataraw[(predictdataraw['orbital_period_[days]'].isnull()) | (predictdataraw['orbit_semi-major_axis_[au])'].isnull()) | (predictdataraw['stellar_radius_[solar_radius]'].isnull())]
len(algo)

212

In [52]:
predictdataraw.drop(algo.index, inplace=True)
predictdataraw['predicted'] = final_pred
predictdataraw

,Unnamed: 0,equilibrium_temperature_[k],orbital_period_[days],orbit_semi-major_axis_[au]),stellar_radius_[solar_radius],insolation_flux_earth,predicted
0,0,NaN,326.030000,1.290000,19.00,NaN,313.321965
1,1,NaN,516.219970,1.530000,29.79,NaN,313.321965
2,2,NaN,185.840000,0.830000,11.00,NaN,313.321965
3,3,NaN,1765.038901,2.773069,0.93,NaN,308.484799
4,4,NaN,798.500000,1.660000,1.13,NaN,311.602502
...,...,...,...,...,...,...,...
1375,4687,NaN,228.776000,0.704800,0.65,NaN,307.441550
1376,4690,NaN,30.000000,0.170000,1.12,NaN,311.602502
1377,4750,NaN,1107.592300,2.720500,1.79,NaN,309.291012
1378,4762,NaN,1919.000000,3.053000,1.10,NaN,302.706461


In [53]:
finaldata = predictdataraw[['Unnamed: 0', 'predicted']]
finaldata

,Unnamed: 0,predicted
0,0,313.321965
1,1,313.321965
2,2,313.321965
3,3,308.484799
4,4,311.602502
...,...,...
1375,4687,307.441550
1376,4690,311.602502
1377,4750,309.291012
1378,4762,302.706461


In [54]:
rows = list(finaldata['Unnamed: 0'])
rows2 = list(finaldata['predicted'])
nextdata = pd.read_csv('nextdata.csv')
nextdata

,Unnamed: 0,planet_name,host_name,orbital_period_[days],orbit_semi-major_axis_[au]),planet_radius,planet_mass,planet_density,eccentricity,equilibrium_temperature_[k],ratio_of_semi-major_axis_to_stellar_radius,stellar_effective_temperature_[k],stellar_radius_[solar_radius],stellar_mass_[solar_mass],stellar_metallicity_[dex],distance_[pc],number_of_planets,insolation_flux_earth,stellar_surface_gravity,stellar_age_[gyr]
0,0,11 Com b,11 Com,326.030000,1.290000,12.10,6165.60000,19.100000,0.231000,,14.599543,4742.00,19.00,2.70,-0.350,93.1846000,1,,2.31,
1,1,11 UMi b,11 UMi,516.219970,1.530000,12.30,4684.81420,13.800000,0.080000,,11.043941,4213.00,29.79,2.78,-0.020,125.3210000,1,,1.93,1.560
2,2,14 And b,14 And,185.840000,0.830000,12.90,1525.50000,3.900000,0.000000,,16.225144,4813.00,11.00,2.20,-0.240,75.4392000,1,,2.63,4.500
3,3,14 Her b,14 Her,1765.038901,2.773069,12.60,2586.74645,7.110000,0.372689,,641.181348,5338.00,0.93,0.91,0.405,17.9323000,2,,4.45,3.900
4,4,16 Cyg B b,16 Cyg B,798.500000,1.660000,13.50,565.73740,1.260000,0.680000,,315.887759,5750.00,1.13,1.08,0.060,21.1397000,1,,4.36,7.400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5260,5260,Kepler-228 d,Kepler-228,11.094286,0.101000,4.04,15.40000,1.280000,0.000000,835,10.900000,6043.00,1.01,0.95,-0.030,1644.6100000,3,122.78,4.47,0.300
5261,5261,Kepler-229 b,Kepler-229,6.252972,0.062000,2.20,5.48000,2.830000,0.000000,823,18.940000,5120.00,0.73,0.80,0.160,826.2650000,3,76.22,4.63,1.100
5262,5262,Kepler-229 c,Kepler-229,16.068638,0.117000,4.92,21.50000,0.992000,0.000000,600,36.850000,5120.00,0.73,0.80,0.160,826.2650000,3,21.64,4.63,1.100
5263,5263,Kepler-229 d,Kepler-229,41.194912,0.220000,3.85,14.20000,1.370000,0.000000,439,70.750000,5120.00,0.73,0.80,0.160,826.2650000,3,6.16,4.63,1.100


In [55]:
filtered = nextdata[nextdata['Unnamed: 0'].isin(rows)]
filtered

,Unnamed: 0,planet_name,host_name,orbital_period_[days],orbit_semi-major_axis_[au]),planet_radius,planet_mass,planet_density,eccentricity,equilibrium_temperature_[k],ratio_of_semi-major_axis_to_stellar_radius,stellar_effective_temperature_[k],stellar_radius_[solar_radius],stellar_mass_[solar_mass],stellar_metallicity_[dex],distance_[pc],number_of_planets,insolation_flux_earth,stellar_surface_gravity,stellar_age_[gyr]
0,0,11 Com b,11 Com,326.030000,1.290000,12.10000,6165.60000,19.100000,0.231000,,14.599543,4742.00,19.00,2.70,-0.350,93.1846000,1,,2.31,
1,1,11 UMi b,11 UMi,516.219970,1.530000,12.30000,4684.81420,13.800000,0.080000,,11.043941,4213.00,29.79,2.78,-0.020,125.3210000,1,,1.93,1.560
2,2,14 And b,14 And,185.840000,0.830000,12.90000,1525.50000,3.900000,0.000000,,16.225144,4813.00,11.00,2.20,-0.240,75.4392000,1,,2.63,4.500
3,3,14 Her b,14 Her,1765.038901,2.773069,12.60000,2586.74645,7.110000,0.372689,,641.181348,5338.00,0.93,0.91,0.405,17.9323000,2,,4.45,3.900
4,4,16 Cyg B b,16 Cyg B,798.500000,1.660000,13.50000,565.73740,1.260000,0.680000,,315.887759,5750.00,1.13,1.08,0.060,21.1397000,1,,4.36,7.400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4687,4687,Kepler-16 b,Kepler-16,228.776000,0.704800,8.44900,105.83300,0.964000,0.006900,,233.160883,4450.00,0.65,0.69,-0.300,75.0852000,1,,4.65,3.800
4690,4690,Kepler-160 d,Kepler-160,30.000000,0.170000,55.25309,100.00000,NaN,,,32.638790,5471.00,1.12,0.97,0.140,937.0130000,3,,4.47,1.600
4750,4750,Kepler-1647 b,Kepler-1647,1107.592300,2.720500,11.87400,483.00000,1.590000,0.058100,,326.812645,6210.00,1.79,1.22,-0.140,1212.4500000,1,,4.02,4.400
4762,4762,Kepler-1656 c,Kepler-1656,1919.000000,3.053000,14.00000,126.40000,0.253000,0.527000,,596.811622,5569.00,1.10,1.03,0.190,185.8750000,2,,4.40,6.310


In [56]:
#pon aqui 
nextdata.loc[filtered.index, 'equilibrium_temperature_[k]'] = rows2
nextdata['equilibrium_temperature_[k]'] = nextdata['equilibrium_temperature_[k]'].replace(' ', np.nan)
nextdata['equilibrium_temperature_[k]'] = nextdata['equilibrium_temperature_[k]'].astype(float)

In [57]:
nextdata.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5265 entries, 0 to 5264
Data columns (total 20 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   Unnamed: 0                                  5265 non-null   int64  
 1   planet_name                                 5265 non-null   object 
 2   host_name                                   5265 non-null   object 
 3   orbital_period_[days]                       5261 non-null   float64
 4   orbit_semi-major_axis_[au])                 5076 non-null   float64
 5   planet_radius                               5260 non-null   float64
 6   planet_mass                                 5260 non-null   float64
 7   planet_density                              5234 non-null   float64
 8   eccentricity                                5265 non-null   object 
 9   equilibrium_temperature_[k]                 4867 non-null   float64
 10  ratio_of_sem

In [58]:
nextdata.drop(['Unnamed: 0'], axis=1, inplace=True)
nextdata.to_csv('nextdata.csv')

In [59]:
#pon aqui
nextdata.to_csv('equilibrium_temperature_[k]_backup.csv')

In [1]:
5265 - 4867

398